In [1]:
import requests
import csv, re, os
import json
import pandas as pd
import numpy as np
from time import sleep

In [2]:
answers = pd.read_csv('weibo/weibo.csv',dtype=object)
savefile = 'weibo/weibo_comments.csv'
savefile_root = 'weibo/weibo_root_count.csv'

In [3]:
with open(savefile, 'a+') as fc open(savefile_root, 'a+') as fr:
    header_fc = ['answer_id','answer_content','comment_id','created_time','comment_content','comment_upvote',
              'child_comments_count','author_info','author_gender']
    writer_fc = csv.DictWriter(fc, delimiter=',', lineterminator='\n',fieldnames=header)
    if not os.path.getsize(savefile):
        writer_fc.writeheader()

    header_fr = ['answer_id','root_comment']
    writer_fr = csv.DictWriter(fr, delimiter=',', lineterminator='\n',fieldnames=header)
    if not os.path.getsize(savefile_root):
        writer_fr.writeheader()

    for index, row in weibos.iterrows():
        page = 0
        root_comment = 0
        while True:
            start_url = f'https://m.weibo.cn/comments/hotflow?id={mid}&mid={mid}&max_id_type={page}'
            try:
                res = json.loads(requests.get(start_url).text)
            except:
                break
            for d in res['data']['data']:
                comment_id = d['id']
                comment_content = re.sub('<.+>', '', d['text'])
                author_location = d['source'].replace('来自','')
                author_info = d['user']['profile_url']
                author_gender = d['user']['gender']
                author_following = d['user']['follow_count']
                author_followers = d['user']['followers_count']
                child_comments_count = d['total_number']
                comment_like = d['like_count']
                writer_fc.writerow({'answer_id':row['mid'], 'answer_content':row['text'],'comment_id':comment_id,
                                 'created_time':d['created_at'],'comment_content':comment_content,'comment_upvote':comment_like,
                                 'child_comments_count':child_comments_count,'author_info':author_info,'author_gender':author_gender,
                                 'author_location':author_location,'author_following':author_following,'author_followers':author_followers})
            page += 1
            root_comment += len(res['data']['data'])
            sleep(random.uniform(3, 5))
        writer_fr.writerow('answer_id':row['mid'],'root_comment':root_comment)

SyntaxError: invalid syntax (390691622.py, line 1)

In [ ]:
import csv

# Define a function to extract the root comment from the mid column
def get_root_comment(mid):
    # Your logic to extract the root comment from mid column here
    root_comment = mid[0:4]
    return root_comment

# Open the input and output CSV files
with open('weibo/weibo_copy-Copy1.csv', 'r') as input_file, open('weibo/weibo_copy-Copy2.csv', 'w', newline='') as output_file:
    # Create CSV reader and writer objects
    reader = csv.reader(input_file)
    writer = csv.writer(output_file)

    # Write the header row to the output file
    header = next(reader)
    header.append('root_comment')
    writer.writerow(header)

    # Iterate over each row in the input file
    for row in reader:
        # Extract the value of the 'mid' column
        mid = row[0]

        # Generate the root comment based on the mid value
        root_comment = get_root_comment(mid)

        # Append the root comment value to the row
        row.append(root_comment)

        # Write the updated row to the output file
        writer.writerow(row)
